<a href="https://colab.research.google.com/github/julianovale/simulacao_python/blob/master/0008_ex_trem_kronecker_artigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from sympy import I, Matrix, symbols, Symbol, eye
from datetime import datetime
import numpy as np
import pandas as pd


In [32]:
'''
Rotas
'''
R1 = Matrix([[0,"p3",0,0,0,0],[0,0,"v1",0,0,0],[0,0,0,"p4",0,0],[0,0,0,0,"v3",0],[0,0,0,0,"v4",0],[0,0,0,0,0,0]])
R2 = Matrix([[0,"p3",0,0,0,0],[0,0,"v2",0,0,0],[0,0,0,"p5",0,0],[0,0,0,0,"v3",0],[0,0,0,0,"v5",0],[0,0,0,0,0,0]])
R3 = Matrix([[0,"p3",0,0,0,0],[0,0,"v5",0,0,0],[0,0,0,"p1",0,0],[0,0,0,0,"v3",0],[0,0,0,0,"v1",0],[0,0,0,0,0,0]])

In [33]:
'''
Seções de bloqueio
'''
T1 = Matrix([[0, 'p1'],['v1', 0]])
T2 = Matrix([[0, 'p2'],['v2', 0]])
T3 = Matrix([[0, 'p3'],['v3', 0]])
T4 = Matrix([[0, 'p4'],['v4', 0]])
T5 = Matrix([[0, 'p5'],['v5', 0]])

In [34]:
momento_inicio = datetime.now() 
'''
Algebra
'''
rotas = np.kron(np.kron(R1, eye(6))+np.kron(eye(6), R2), eye(6))+np.kron(eye(36),R3)
secoes = np.kron(np.kron(np.kron((np.kron(T1, eye(2)) + np.kron(eye(2), T2)),eye(2))+np.kron(eye(4),T3), eye(2))+np.kron(eye(8), T4), eye(2))+np.kron(eye(16),T5)
sistema = np.kron(rotas, secoes)

# calcula tempo de processamento
tempo_processamento = datetime.now() - momento_inicio

In [35]:
sistema = pd.DataFrame(sistema)

In [36]:
sistema.shape

(6912, 6912)

In [37]:
print(tempo_processamento)

0:01:11.158743


In [39]:
sistema

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,6872,6873,6874,6875,6876,6877,6878,6879,6880,6881,6882,6883,6884,6885,6886,6887,6888,6889,6890,6891,6892,6893,6894,6895,6896,6897,6898,6899,6900,6901,6902,6903,6904,6905,6906,6907,6908,6909,6910,6911
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,p3*p5,p3*p4,0,p3**2,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,p3*v5,0,0,p3*p4,0,p3**2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,p3*v4,0,0,p3*p5,0,0,p3**2,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,p3*v4,p3*v5,0,0,0,0,p3**2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,p3*v3,0,0,0,0,p3*p5,p3*p4,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6908,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6909,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6910,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
